In [ ]:
%load_ext autoreload
%autoreload 2

import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre

import os
import sys
import mne
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import multiprocessing as mpr
import matplotlib as mpl
import time
import gc;
import scipy.signal as sig
import pandas as pd 

import numpy as np
import utils_postprocess_HPC as postp

data_dir = gv.data_dir
from os.path import join as pjoin

In [ ]:
subdir = ''
fname_full_summary = pjoin(gv.data_dir,subdir,'output_per_raw.npz')
f = np.load(fname_full_summary,allow_pickle=True)

In [ ]:
output_per_raw = f['output_per_raw']

output_per_raw = output_per_raw[()]

In [ ]:
output_per_raw = output_per_raw[()]

In [ ]:
prefixes = list( sorted(output_per_raw['S01'].keys() ) )
prefixes

In [ ]:
perf_to_use_list = []
perf_to_use_list += [('XGB','perfs_XGB','perfs_XGB_red') ]
perf_to_use_list += [('XGB','perfs_XGB','perfs_XGB_fs_red') ]
perf_to_use_list += [('LDA','all_present_features','strongest_features_LDA_opinion') ]
perf_to_use_list += [('LDA','all_present_features','strongest_features_LDA_selMinFeatSet') ]
# these are actually XGB_selMeanFeatSet
perf_to_use_list += [('LDA','all_present_features','strongest_features_XGB_opinion') ]
perf_to_use_list += [('LDA','all_present_features','strongest_features_XGB_fs_opinion') ]


all_LDA =  ['all_present_features',
      'best_PCA-derived_features_0.6',
      'best_PCA-derived_features_0.75',
      'best_PCA-derived_features_0.9',
      'strongest_features_LDA_opinion',
      'strongest_features_LDA_selMinFeatSet',
      'strongest_features_XGB_opinion',
      'strongest_features_XGB_fs_opinion' ]

all_XGB = ['all_present_features',
      'strongest_features_XGB_opinion',
      'strongest_features_XGB_opinion_nosyn',
      'best_PCA-derived_features_0.6',
      'best_PCA-derived_features_0.75',
      'best_PCA-derived_features_0.9',
      'strongest_features_XGB_fs_opinion',
      'strongest_features_XGB_fs_opinion_nosyn']

perf_to_use_list = []
for v in all_LDA[1:]:
    perf_to_use_list += [('LDA',all_LDA[0],v)]
for v in all_XGB[1:]:
    perf_to_use_list += [('XGB',all_XGB[0],v)]
    
    
    
to_show = [('allsep','merge_nothing','basic'), ('trem_vs_all','merge_all_not_trem','basic'),
        ('trem_vs_2class','merge_movements','basic')]

# warnings.simplefilter('error')
# table_info_per_perf_type, table_per_perf_type = \
#     postp.prepTableInfo2(output_per_raw, prefixes=prefixes, 
#     perf_to_use_list=perf_to_use_list)

#%debug
# import warnings
# with warnings.catch_warnings():
#warnings.simplefilter('error')
table_info_per_perf_type, table_per_perf_type = \
    postp.prepTableInfo3(output_per_raw, prefixes=prefixes, 
    perf_to_use_list=perf_to_use_list, to_show=to_show)

In [ ]:
for perf_tuple in perf_to_use_list:
    print(perf_tuple)
    postp.plotTableInfos2(table_info_per_perf_type, perf_tuple=perf_tuple, 
                          output_subdir='') 

In [ ]:
import utils_postprocess as pp
pp.printDict(output_per_raw,max_depth=2)

In [ ]:
prefixes

In [ ]:
prefix = 'all'
prefix = 'LFPrel_noself'
prefix = 'LFPrel_noself_onlyBpcorr'
# prefix = 'LFPrel_noself_onlyBpcorr_onlyMotorSrc'
# prefix = 'cross_freqmod_beta,gamma:HFO'
# prefix = 'onlyH'
# prefix = 'modSrc'
# prefix = 'LFPrel_noself_onlyBpcorr_onlyRestSrc'
#for prefix in prefixes:
#feat_inds_per_prefix = postp.plotFeatNum2Perf(output_per_raw, prefixes)

perflists = ['perfs_XGB','perfs_XGB_fs', 'perfs_LDA_featsearch']

prefix_list =  ['all', 'LFPrel_noself', 'LFPrel_noself_onlyBpcorr', 'onlyH']
#prefix_list = ['LFPrel_noself_onlyRbcorr']
#prefix_list =  [ 'LFPrel_noself', 'LFPrel_noself_onlyBpcorr', 'onlyH']
feat_names_per_prefix = postp.plotFeatNum2Perf(output_per_raw, perflists, prefix_list,
                                              skip_plot=1)

In [ ]:
feat_names_per_prefix

In [ ]:
np.hstack( [np.zeros((7, 3)), np.zeros((7,1))] ).shape


In [ ]:
feat_names_per_prefix[prefix][rn][(g,it_set)].keys()

In [ ]:
#%debug
perflists = ['perfs_XGB','perfs_XGB_fs', 'perfs_LDA_featsearch'] [:1]
for pt in perflists: 
    for prefix in feat_names_per_prefix.keys():
        #pt = 'perfs_XGB'
        featlist_per_rn={}
        rns = list(sorted(output_per_raw.keys()))
        rowind = 0
        for rn in rns:

            feat_names_per_pg_piset = {}
            #for prefix,pg in output_per_raw[rn].items():
            pg = output_per_raw[rn][prefix]
            for g,pitset in pg.items():
                if g == 'feature_names_filtered':
                    continue
                for it_set,multi_clf_output in pitset.items():

                    featnames = feat_names_per_prefix[prefix][rn][(g,it_set)][pt]
                    if featnames is None:
                        continue
                    postp.plotImportantFeatLocations(rn,multi_clf_output,featnames)
                    figname = f'important_feat_headloc__{prefix}_{rn}_{g,it_set}__{pt}.pdf'
                    plt.suptitle(f'Number of (balanced) features = {len(featnames)}')
                    plt.savefig(pjoin(gv.dir_fig, figname ) )

#                     break
#                 break
#             break
#         break

In [ ]:
sind_str=rn; color_by_ftype=True; head_subj_ind='S01'
    # featnames here are NOT nice, but it is already a subset, not full set



In [ ]:
A = [5,6,8]; inds = [0, 1, 0 ,1 ,0, 1, 2, 0,0]
np.array(A)[inds]

In [ ]:
MI = multi_clf_output['MI_per_feati']
plt.hist( MI )

In [ ]:
q = 0.8
MIq = np.quantile(MI, q)
print(MIq)
np.where(MI > MIq)

In [ ]:
C = f['results_light']  [()]['corr_matrix']

len(fip_fs)

## Throw away correlated features

indlist = fip_fs
#indlist = np.arange(C.shape[0])
C_subset = C[indlist,:][:,indlist]
strong_correl_level = 0.85

nonsyn_feat_inds = pp.getNotSyn(C_subset,strong_correl_level)
#orig_inds = indlist[nonsyn_feat_inds]

print(f'Without synonyms for corr. level = {strong_correl_level:.3f}: {len(nonsyn_feat_inds)} of {len(indlist)}' )
plt.hist(C_subset.flatten());
plt.pcolor(C_subset)

In [ ]:
featnames

In [ ]:

 
r2['mod_per_ftype']

In [ ]:
None == []

In [ ]:
# labels

In [ ]:
featnames

In [ ]:
sizes

In [ ]:
r['mod1'],r['mod2']

In [ ]:
r = parseFeatNames(featnames); r

In [ ]:
codes

In [ ]:
tuples_str

In [ ]:
codes2

In [ ]:
codes_present

In [ ]:
# maybe I want to rename the codes so that they are from 0 to len(set(codes))
# convert tuples to string_labels
# maybe use '_' instead of None for faster conversion

In [ ]:
codes

In [ ]:
r.keys()

In [ ]:
ur = {}
for k,item in r.items():
    ur[k] = list( sorted( set(item) ) )
    
display(ur)
# ftypes = np.sort(np.unique( r['ftype'] ) )
# mod1 = np.sort(np.unique( r['mod1'] ) )
# mod2 = np.sort(np.unique( r['mod2'] ) )
# fb1 = np.sort(np.unique( r['2'] ) )
# fb2 = np.sort(np.unique( r['mod2'] ) )

In [ ]:
#%debug
rname_crop = slice(0,3)
for rn in output_per_raw:
    for lt, it_grp, it_set in to_show:
    #for lt in label_types:
        info_per_pref = {}
        # raw and label type (grouping+int_types) name goes here
        # this will be a row name
        row_name = '{}_{}'.format(rn[rname_crop], lt)
        table_row = [row_name ]
        for prefix in prefixes:
            info_cur = {}
            #sens,spec = res[rn].get(pref, (np.nan, np.nan))
            r = output_per_raw[rn].get(prefix, None)
            if (r is not None) and (it_grp not in r or it_set not in r[it_grp]):
                r = None

            if r is None:
                print('Warning : no output for ',rn,prefix,it_grp,it_set)
                continue
            else:
                lda_output = r[it_grp][it_set]

            use_light_file = 1
            if use_light_file:
                featsel_per_method = lda_output['featsel_per_method'] 
                featnames = lda_output['feature_names_filtered']
                class_labels_good = lda_output['class_labels_good']
            else:                
                f = np.load( lda_output['filename_full'], allow_pickle=True )
                featnames =  f['feature_names_filtered_pri'][()][0]
                print(prefix, f['Xconcat_good_cur'].shape, len(featnames) )
                class_labels_good = f['class_labels_good']
                
            revdict = lda_output['revdict']
            from sklearn import preprocessing
            lab_enc = preprocessing.LabelEncoder()
            # just skipped class_labels_good
            lab_enc.fit(class_labels_good)
            class_labels_good_for_classif = lab_enc.transform(class_labels_good)
            class_label_ids = lab_enc.inverse_transform( np.arange( len(set(class_labels_good_for_classif)) ) )
            class_label_names = [revdict[cli] for cli in class_label_ids]
            
            output_subdir = ''
            out_name_plot = f'{rn}_{it_grp},{it_set}_{prefix}_feat_signif'
            from matplotlib.backends.backend_pdf import PdfPages
            mpl.use('Agg')            
            
            try:
                pdf= PdfPages(pjoin(gv.dir_fig, output_subdir, out_name_plot + '.pdf' ))    
                postp.plotFeatSignifSHAP(pdf,featsel_per_method, 'XGB_Shapley', 
                                         featnames, class_labels_good_for_classif,class_label_names,prefix);
                pdf.close()
            except (ValueError) as e:
                print('__EERRRRROOORRR ', e,out_name_plot)
            plt.close('all')
            
            #break
        #break
    #break

In [ ]:
plotFeatImpStats(feat_types_all, feat_imp_stats)